In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
%matplotlib inline

In [2]:
df = pd.read_csv("powerplants.csv")
df.head()

,Plant_Code,Plant_Name,Utility_Na,Utility_ID,sector_nam,City,County,Zip,Street_Add,PrimSource,...,Solar_MW,Wind_MW,Geo_MW,Other_MW,source_des,tech_desc,Source,Period,Latitude,Longitude
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,35476,19001 Lock 17 Road,hydroelectric,...,0.0,0.0,NaN,NaN,Hydroelectric = 56 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",201607,33.458665,-87.356823
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,36512,North Highway 43,coal,...,0.0,0.0,NaN,NaN,"Coal = 1088.5 MW, Natural Gas = 1072 MW",Conventional Steam Coal; Natural Gas Fired Com...,"EIA-860, EIA-860M and EIA-923",201607,31.006900,-88.010300
2,4,Walter Bouldin Dam,Alabama Power Co,195,Electric Utility,Wetumpka,Elmore,36092,750 Bouldin Dam Road,hydroelectric,...,0.0,0.0,NaN,NaN,Hydroelectric = 225.9 MW,Conventional Hydroelectric,"EIA-860, EIA-860M and EIA-923",201607,32.583889,-86.283056
3,7,Gadsden,Alabama Power Co,195,Electric Utility,Gadsden,Etowah,35903,1000 Goodyear Ave,natural gas,...,0.0,0.0,NaN,NaN,Natural Gas = 130 MW,Natural Gas Steam Turbine,"EIA-860, EIA-860M and EIA-923",201607,34.012800,-85.970800
4,8,Gorgas,Alabama Power Co,195,Electric Utility,Parrish,Walker,35580,460 Gorgas Road,coal,...,0.0,0.0,NaN,NaN,Coal = 1034 MW,Conventional Steam Coal,"EIA-860, EIA-860M and EIA-923",201607,33.644344,-87.196486


In [3]:
states = gpd.read_file("cb_2016_us_state_500k/cb_2016_us_state_500k.shp")
states.head()

,AFFGEOID,ALAND,AWATER,GEOID,LSAD,NAME,STATEFP,STATENS,STUSPS,geometry
0,0400000US01,131173688951,4593686489,01,00,Alabama,01,01779775,AL,"(POLYGON ((-88.053375 30.506987, -88.051087999..."
1,0400000US02,1477946266785,245390495931,02,00,Alaska,02,01785533,AK,"(POLYGON ((-134.737262 58.261354, -134.73441 5..."
2,0400000US04,294198560125,1027346486,04,00,Arizona,04,01779777,AZ,"POLYGON ((-114.816294 32.508038, -114.814321 3..."
3,0400000US05,134771517596,2960191698,05,00,Arkansas,05,00068085,AR,"POLYGON ((-94.6178329666013 36.4994141203285, ..."
4,0400000US06,403501101370,20466718403,06,00,California,06,01779778,CA,"(POLYGON ((-118.604415 33.478552, -118.598783 ..."


In [4]:
points = df.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)
powerplants = gpd.GeoDataFrame(df, geometry=points)
powerplants.crs = {'init' : 'epsg:4326'}
states = states.to_crs(powerplants.crs)

In [5]:
powerplants_with_states = gpd.sjoin(powerplants, states, how='left', op='within')
powerplants_with_states.head()

,Plant_Code,Plant_Name,Utility_Na,Utility_ID,sector_nam,City,County,Zip,Street_Add,PrimSource,...,index_right,AFFGEOID,ALAND,AWATER,GEOID,LSAD,NAME,STATEFP,STATENS,STUSPS
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,35476,19001 Lock 17 Road,hydroelectric,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,36512,North Highway 43,coal,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
2,4,Walter Bouldin Dam,Alabama Power Co,195,Electric Utility,Wetumpka,Elmore,36092,750 Bouldin Dam Road,hydroelectric,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
3,7,Gadsden,Alabama Power Co,195,Electric Utility,Gadsden,Etowah,35903,1000 Goodyear Ave,natural gas,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
4,8,Gorgas,Alabama Power Co,195,Electric Utility,Parrish,Walker,35580,460 Gorgas Road,coal,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL


In [6]:
powerplants_with_states['NAME'].value_counts()

California              1222
North Carolina           461
Texas                    396
New York                 372
Minnesota                274
Massachusetts            254
Iowa                     222
Michigan                 219
New Jersey               207
Pennsylvania             207
Illinois                 200
Wisconsin                174
Colorado                 153
Ohio                     148
Oregon                   145
Florida                  144
Georgia                  139
Washington               138
Alaska                   138
Idaho                    136
Indiana                  133
Virginia                 131
Kansas                   129
Missouri                 117
Arizona                  113
Nebraska                  98
Maine                     97
South Carolina            95
Oklahoma                  94
New Mexico                92
Utah                      89
Connecticut               88
Louisiana                 84
Maryland                  76
Vermont       

In [ ]:
#Which power plants are in New York?

In [7]:
powerplants_in_ny = powerplants_with_states[powerplants_with_states['NAME'] == 'New York']
powerplants_in_ny.head()

,Plant_Code,Plant_Name,Utility_Na,Utility_ID,sector_nam,City,County,Zip,Street_Add,PrimSource,...,index_right,AFFGEOID,ALAND,AWATER,GEOID,LSAD,NAME,STATEFP,STATENS,STUSPS
49,88,Ashokan,New York Power Authority,15296,Electric Utility,Kingston,Ulster,12401,Route 28A,hydroelectric,...,43,0400000US36,1.220530e+11,1.924310e+10,36,00,New York,36,01779796,NY
363,579,Central Hudson High Falls,Central Hudson Gas & Elec Corp,3249,Electric Utility,High Falls,ULSTER,12440,Route 213,hydroelectric,...,43,0400000US36,1.220530e+11,1.924310e+10,36,00,New York,36,01779796,NY
387,625,Upper Mechanicville,New York State Elec & Gas Corp,13511,Electric Utility,Town of Stillwater,Saratoga,12170,40 Hudson Avenue,hydroelectric,...,43,0400000US36,1.220530e+11,1.924310e+10,36,00,New York,36,01779796,NY
469,808,Jarvis (Hinckley),New York Power Authority,15296,Electric Utility,Hinckley,Oneida,13352,None,hydroelectric,...,43,0400000US36,1.220530e+11,1.924310e+10,36,00,New York,36,01779796,NY
1169,2480,Danskammer Generating Station,Danskammer Energy,58971,IPP Non-CHP,Newsburgh,Orange,12550,994 River Road,natural gas,...,43,0400000US36,1.220530e+11,1.924310e+10,36,00,New York,36,01779796,NY


In [8]:
powerplants_in_ny['Plant_Name']

49                                  Ashokan
363               Central Hudson High Falls
387                     Upper Mechanicville
469                       Jarvis (Hinckley)
1169          Danskammer Generating Station
1170                              Dashville
1171                              Neversink
1172                            South Cairo
1173                               Sturgeon
1174                         West Coxsackie
1175         Arthur Kill Generating Station
1176                             East River
1177        Gowanus Gas Turbines Generating
1178                          Hudson Avenue
1179                         Indian Point 2
1180        Narrows Gas Turbines Generating
1181                             Ravenswood
1182                            59th Street
1183                            74th Street
1184                            E F Barrett
1185                           East Hampton
1186                               Glenwood
1187                            

In [ ]:
# Which state has the most hydroelectric power plants?

In [9]:
hydroelectric = powerplants_with_states[powerplants_with_states['PrimSource'] == 'hydroelectric']
hydroelectric.head()

,Plant_Code,Plant_Name,Utility_Na,Utility_ID,sector_nam,City,County,Zip,Street_Add,PrimSource,...,index_right,AFFGEOID,ALAND,AWATER,GEOID,LSAD,NAME,STATEFP,STATENS,STUSPS
0,2,Bankhead Dam,Alabama Power Co,195,Electric Utility,Northport,Tuscaloosa,35476,19001 Lock 17 Road,hydroelectric,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
2,4,Walter Bouldin Dam,Alabama Power Co,195,Electric Utility,Wetumpka,Elmore,36092,750 Bouldin Dam Road,hydroelectric,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
7,11,H Neely Henry Dam,Alabama Power Co,195,Electric Utility,Ohatchee,Calhoun,36271,1021 Ala Highway 144,hydroelectric,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
8,12,Holt Dam,Alabama Power Co,195,Electric Utility,Tuscaloosa,Tuscaloosa,35476,12117 Power Plant Road,hydroelectric,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
9,13,Jordan Dam,Alabama Power Co,195,Electric Utility,Wetumpka,Elmore,36092,6150Jordan Dam Lane,hydroelectric,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL


In [10]:
hydroelectric['NAME'].value_counts()

California        245
New York          159
Washington         77
Idaho              75
Oregon             60
Wisconsin          59
Michigan           53
Maine              52
Colorado           43
Vermont            42
North Carolina     41
New Hampshire      38
South Carolina     31
Utah               29
Georgia            29
Massachusetts      28
Tennessee          27
Alabama            23
Alaska             23
Montana            23
Virginia           23
Texas              22
Minnesota          21
Arkansas           19
Wyoming            16
Pennsylvania       16
Connecticut        13
West Virginia      12
Oklahoma           10
Nebraska           10
Kentucky            9
Illinois            8
Hawaii              7
Arizona             7
Nevada              7
Indiana             6
Missouri            5
Ohio                4
New Mexico          4
South Dakota        4
Maryland            3
Rhode Island        2
New Jersey          2
Iowa                2
Florida             2
North Dako

In [ ]:
# Which state produces the most energy from coal?

In [11]:
coal = powerplants_with_states[powerplants_with_states['PrimSource'] == 'coal']
coal.head()

,Plant_Code,Plant_Name,Utility_Na,Utility_ID,sector_nam,City,County,Zip,Street_Add,PrimSource,...,index_right,AFFGEOID,ALAND,AWATER,GEOID,LSAD,NAME,STATEFP,STATENS,STUSPS
1,3,Barry,Alabama Power Co,195,Electric Utility,Bucks,Mobile,36512,North Highway 43,coal,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
4,8,Gorgas,Alabama Power Co,195,Electric Utility,Parrish,Walker,35580,460 Gorgas Road,coal,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
18,26,E C Gaston,Alabama Power Co,195,Electric Utility,Wilsonville,Shelby,35186,31972 Highway 25 North,coal,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
23,47,Colbert,Tennessee Valley Authority,18642,Electric Utility,Tuscumbia,Colbert,35674,900 Steam Plant Rd.,coal,...,0,0400000US01,1.311737e+11,4.593686e+09,01,00,Alabama,01,01779775,AL
26,51,Dolet Hills,Cleco Power LLC,3265,Electric Utility,Mansfield,DeSoto,71052,963 Power Plant Rd,coal,...,36,0400000US22,1.119049e+11,2.374630e+10,22,00,Louisiana,22,01629543,LA


In [12]:
coal['NAME'].value_counts()

Pennsylvania      24
Illinois          22
Texas             21
Indiana           20
Iowa              18
Michigan          16
Missouri          16
Ohio              16
Kentucky          14
North Carolina    13
Wyoming           12
Wisconsin         12
Minnesota         12
West Virginia     11
Florida           10
Colorado          10
Nebraska          10
North Dakota      10
Virginia          10
South Carolina     8
Maryland           7
Tennessee          6
Alabama            6
Oklahoma           6
Utah               6
Georgia            6
Kansas             6
Arkansas           5
Arizona            5
New York           5
Montana            5
New Jersey         4
Alaska             4
New Mexico         3
Nevada             3
Mississippi        3
Louisiana          2
New Hampshire      2
California         2
Idaho              2
Massachusetts      1
Washington         1
Hawaii             1
South Dakota       1
Delaware           1
Oregon             1
Connecticut        1
Name: NAME, d